#### Join all features -> train/test set -> FFM input format
- **This notebook only select a sample uuid hash from 0 - 9**
- 1 Join events_uusum feaures onto f4_all
- 2 Encode categorical features with int id
- 3 Transform into FFM input format
    - 3.1 [add-on] Combine/remove infrequent categorical values
- 4 Select a sample and train/test FFM, output validation score in logloss and map

In [1]:
#### import modules ####
import pandas as pd
import numpy as np
import time, gc, logging, argparse
from scipy import sparse
from itertools import combinations

In [2]:
#### Read f4_all ####
f4_all = pd.read_csv("../interm/f4_all.csv.gz", header = 0)

In [3]:
#### Check f4_all ####
print f4_all.shape
f4_all.head()

(237183763, 23)


,display_id,ad_id,clicked,uuid,events:document_id,events:timestamp,events:platform,events:geo_location,ad_prom:document_id,ad_prom:campaign_id,...,ad_prom:document_id:publish_time,events:document_id:source_id,events:document_id:publisher_id,events:document_id:publish_time,events:document_id:topic_topI,events:document_id:entity_topI,events:document_id:categ_topI,ad_prom:document_id:topic_topI,ad_prom:document_id:entity_topI,ad_prom:document_id:categ_topI
0,1,42337,0,cb8c55702adb93,379743,2016-06-14 04:00:00,3,US>SC>519,938164,5969,...,2016-07-27 00:00:00,6482.0,24.0,NaN,285,-99,1203,NaN,NaN,NaN
1,1,42337,0,cb8c55702adb93,379743,2016-06-14 04:00:00,3,US>SC>519,938164,5969,...,2016-07-27 00:00:00,6482.0,24.0,NaN,285,-99,1203,NaN,NaN,NaN
2,1,139684,0,cb8c55702adb93,379743,2016-06-14 04:00:00,3,US>SC>519,1085937,17527,...,2016-02-09 15:00:00,6482.0,24.0,NaN,285,-99,1203,NaN,NaN,NaN
3,1,139684,0,cb8c55702adb93,379743,2016-06-14 04:00:00,3,US>SC>519,1085937,17527,...,2016-02-09 15:00:00,6482.0,24.0,NaN,285,-99,1203,NaN,NaN,NaN
4,1,144739,1,cb8c55702adb93,379743,2016-06-14 04:00:00,3,US>SC>519,1337362,18488,...,2016-04-24 08:00:00,6482.0,24.0,NaN,285,-99,1203,NaN,NaN,NaN


In [5]:
####### f4_all has duplicated display-ad, remove to see if it matches train and test ####
print f4_all.loc[:,['display_id','ad_id']].drop_duplicates().shape
f4_all_notdup = f4_all.drop_duplicates(['display_id','ad_id'])
print f4_all_notdup.shape
clicks_train = pd.read_csv('../data/clicks_train.csv.gz', header = 0)
clicks_test = pd.read_csv('../data/clicks_test.csv.gz', header = 0)

    

(119366893, 2)
           display_id   ad_id  clicked            uuid  events:document_id  \
0                   1   42337        0  cb8c55702adb93              379743   
2                   1  139684        0  cb8c55702adb93              379743   
4                   1  144739        1  cb8c55702adb93              379743   
6                   1  156824        0  cb8c55702adb93              379743   
8                   1  279295        0  cb8c55702adb93              379743   
10                  1  296965        0  cb8c55702adb93              379743   
12                  2  125211        0  79a85fa78311b9             1794259   
14                  2  156535        0  79a85fa78311b9             1794259   
15                  2  169564        0  79a85fa78311b9             1794259   
17                  2  308455        1  79a85fa78311b9             1794259   
19                  3   71547        0  822932ce3d8757             1179111   
21                  3   95814        0  822932ce3

In [8]:
##### check if the ids match original train/test ####
print clicks_train.shape[0] + clicks_test.shape[0] == 119366893
print set((f4_all_notdup.loc[f4_all_notdup.clicked != -1,'display_id'].astype(str) + '-' + \
            f4_all_notdup.loc[f4_all_notdup.clicked != -1,'ad_id'].astype(str) ).tolist()) ==\
        set((clicks_train.display_id.astype(str) + '-' + clicks_train.ad_id.astype(str) ).tolist())
print set((f4_all_notdup.loc[f4_all_notdup.clicked == -1,'display_id'].astype(str) + '-' + \
            f4_all_notdup.loc[f4_all_notdup.clicked == -1,'ad_id'].astype(str) ).tolist()) ==\
        set((clicks_test.display_id.astype(str) + '-' + clicks_test.ad_id.astype(str) ).tolist())

True
True
True


In [9]:
##### save f4_all_notdup ####
filename_output = "../interm/f4_all_notdup.csv.gz"
f4_all_notdup.to_csv(filename_output, index = False, compression="gzip")

In [10]:
#### Only select a sample in this notebook: uuid 0 - 9 ####
hash_length = 100
def get_hash_code(x):
    y = abs(hash(x)%hash_length)
    return int(y)

f4_all_notdup.loc[:,"uuid_hash_code"] = f4_all_notdup["uuid"].apply(get_hash_code)
f4_all_notdup_uu = f4_all_notdup.loc[(f4_all_notdup.uuid_hash_code >= 0) & (f4_all_notdup.uuid_hash_code <10), :]
del f4_all, f4_all_notdup, clicks_train, clicks_test


/home/ubuntu/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/ubuntu/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:476: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [12]:
gc.collect()
print f4_all_notdup_uu.shape
f4_all_notdup_uu.head()
print f4_all_notdup_uu.uuid_hash_code.unique()

(11918046, 24)
[2 9 0 1 7 5 3 4 8 6]


In [13]:
#### Concatenete events_uusum features into a big table ####
uu_hash_used = ['0-9']
events_uusum_feature = None
for uu_hash in uu_hash_used:
    print uu_hash
    filepath = '../data/events_uusum_feature/events_uusum_feature' + uu_hash + '.csv.gz'
    events_uusum = pd.read_csv(filepath, header = 0)
    if events_uusum_feature is None:
        events_uusum_feature = events_uusum
    else:
        events_uusum_feature = pd.concat([events_uusum_feature,events_uusum], axis=0)
del events_uusum
gc.collect()

0-9


68

In [17]:
#### check events_uusum_feature ####

# events_uusum_feature.head()
new_col_names = {}
for col in events_uusum_feature.columns:
    if col in ['uuid','uuid_hash_code']:
        continue
    if col in ['num_visits','platform-freq','geo_location-freq', 'traffic_source-freq']:
        new_col_names.update({col: 'page_views:uuid:' + col})
    else:
        new_col_names.update({col: 'page_views:document_id:' + col})
events_uusum_feature.rename(columns = new_col_names, inplace = True)
print events_uusum_feature.shape
events_uusum_feature.head()


(1977383, 11)


,uuid,uuid_hash_code,page_views:uuid:num_visits,page_views:uuid:platform-freq,page_views:uuid:geo_location-freq,page_views:uuid:traffic_source-freq,page_views:document_id:source_id-freq,page_views:document_id:publisher_id-freq,page_views:document_id:topic_topI-freq,page_views:document_id:categ_topI-freq,page_views:document_id:entity_topI-freq
0,c419799a427c72,0,3,3:3,US>HI>744:3,1:3,1014:1|2734:2,38:1|863:2,128:1|26:1|297:1,2100:3,6837babadc7ee40652ecdb5399e6a207:1|d43ef0ce148...
1,f8198611130910,0,74,1:74,US>MD>512:74,1:68|2:3|3:3,10198:1|1059:1|148:1|1803:1|1886:1|2686:1|2799...,1046:1|1084:1|1175:1|1221:50|158:1|160:2|180:3...,-99:17|10:1|111:1|125:2|140:5|147:1|15:1|160:1...,-99:9|1403:3|1405:1|1406:1|1407:1|1408:1|1607:...,-99:45|034008df3f5e6cff40da618ee25fef76:1|22af...
2,3abf1630ff91c5,0,8,1:8,CA>BC:8,1:4|2:1|3:3,13858:1|2591:1|5792:1|7734:1|7736:1|822:1|9135...,1042:1|1064:1|202:1|253:1|450:2|704:2,130:1|16:1|170:1|181:1|20:1|49:1|65:1|8:1,1408:1|1510:1|1515:1|1609:1|1610:1|1611:1|1709...,-99:2|3d6c006d132611e4ee097cb5814a42b4:1|7d0cd...
3,a829cd392e8574,0,3,2:3,US>CA>807:3,1:3,1809:3,158:3,66:3,1611:3,504e40cfc426912d179d7e436c4f945b:3
4,5ab105d8e0defb,0,1,1:1,AE>03:1,1:1,8803:1,1157:1,26:1,1403:1,-99:1


In [19]:
#### Join into f5_all ####
f5_all_notdup_uu = f4_all_notdup_uu.merge(events_uusum_feature, how = 'left',on = 'uuid', suffixes = ['','_right'])
print f5_all_notdup_uu.shape
f5_all_notdup_uu.head()

(11918046, 34)


,display_id,ad_id,clicked,uuid,events:document_id,events:timestamp,events:platform,events:geo_location,ad_prom:document_id,ad_prom:campaign_id,...,uuid_hash_code_right,page_views:uuid:num_visits,page_views:uuid:platform-freq,page_views:uuid:geo_location-freq,page_views:uuid:traffic_source-freq,page_views:document_id:source_id-freq,page_views:document_id:publisher_id-freq,page_views:document_id:topic_topI-freq,page_views:document_id:categ_topI-freq,page_views:document_id:entity_topI-freq
0,2,125211,0,79a85fa78311b9,1794259,2016-06-14 04:00:00,2,US>CA>807,876083,16054,...,2,159,2:159,US>CA:6|US>CA>807:151|US>CA>862:2,1:159,6715:19|6716:2|6717:5|6719:4|93:129,407:159,-99:63|10:3|105:1|111:4|116:1|118:1|126:1|131:...,1100:1|1403:125|1405:2|1406:2|1407:3|1408:11|1...,-99:61|01970e2e4c8a3c62b1ede7b0318dc4a8:1|069b...
1,2,156535,0,79a85fa78311b9,1794259,2016-06-14 04:00:00,2,US>CA>807,630534,16636,...,2,159,2:159,US>CA:6|US>CA>807:151|US>CA>862:2,1:159,6715:19|6716:2|6717:5|6719:4|93:129,407:159,-99:63|10:3|105:1|111:4|116:1|118:1|126:1|131:...,1100:1|1403:125|1405:2|1406:2|1407:3|1408:11|1...,-99:61|01970e2e4c8a3c62b1ede7b0318dc4a8:1|069b...
2,2,169564,0,79a85fa78311b9,1794259,2016-06-14 04:00:00,2,US>CA>807,1394819,20109,...,2,159,2:159,US>CA:6|US>CA>807:151|US>CA>862:2,1:159,6715:19|6716:2|6717:5|6719:4|93:129,407:159,-99:63|10:3|105:1|111:4|116:1|118:1|126:1|131:...,1100:1|1403:125|1405:2|1406:2|1407:3|1408:11|1...,-99:61|01970e2e4c8a3c62b1ede7b0318dc4a8:1|069b...
3,2,308455,1,79a85fa78311b9,1794259,2016-06-14 04:00:00,2,US>CA>807,1712989,245,...,2,159,2:159,US>CA:6|US>CA>807:151|US>CA>862:2,1:159,6715:19|6716:2|6717:5|6719:4|93:129,407:159,-99:63|10:3|105:1|111:4|116:1|118:1|126:1|131:...,1100:1|1403:125|1405:2|1406:2|1407:3|1408:11|1...,-99:61|01970e2e4c8a3c62b1ede7b0318dc4a8:1|069b...
4,7,105766,1,2cc3f6457d16da,1149661,2016-06-14 04:00:00,2,US>MT>762,1151028,2176,...,9,13,2:13,US>MT>762:12|US>TX:1,1:2|3:11,3420:1|3453:2|6817:1|7882:2|8616:3|9453:4,1012:3|1149:4|1151:2|384:1|967:1|968:2,103:1|176:1|20:2|214:3|252:3|254:1|261:1|82:1,1602:1|1611:1|1702:1|1703:1|1706:2|1707:2|1708:5,-99:2|0bab6ac17939465fdea63bb79009c4be:1|142c2...


In [20]:
#### save f5_all into file ####
#### This is just a sample ####
f5_all_notdup_uu.to_csv()
filename_output = "../interm/f5_all_notdup0-9.csv.gz"
f5_all_notdup_uu.to_csv(filename_output, index = False, compression="gzip")

In [29]:
#### Modules for sparse onehot encoding ####
def OneHotEncoder(data, keymap=None):
    """
    OneHotEncoder takes data matrix with categorical columns and
    converts it to a sparse binary matrix.
    Returns sparse binary matrix and keymap mapping categories to indicies.
    If a keymap is supplied on input it will be used instead of creating one
    and any categories appearing in the data that are not in the keymap are
    ignored
    """
    if keymap is None:
        keymap = []
        for col in data.T:
            uniques = set(list(col))
            keymap.append(dict((key, i) for i, key in enumerate(uniques)))
    total_pts = data.shape[0]
    outdat = []
    for i, col in enumerate(data.T):
        km = keymap[i]
        num_labels = len(km)
        spmat = sparse.lil_matrix((total_pts, num_labels))
        for j, val in enumerate(col):
            if val in km:
                spmat[j, km[val]] = 1
        outdat.append(spmat)
    outdat = sparse.hstack(outdat).tocsr()           
    return outdat, keymap

def OneHotEncoderMapper(data):
    """
    Only generate mapper from OneHotEncoder
    """
    keymap = None
    if keymap is None:
        keymap = []
        for col in data.T:
            uniques = set(list(col))
            keymap.append(dict((key, i) for i, key in enumerate(uniques)))             
    return keymap

In [26]:
# select feature cols to encode 

print f5_all_notdup_uu.columns.values.tolist()
feature_cols = [x for x in f5_all_notdup_uu.columns.values.tolist() \
                if 'events:' in x or 'ad_prom:' in x or 'page_views:' in x]
feature_cols = [x for x in feature_cols if 'time' not in x and 'num_visits' not in x]
feature_cols
n_feat = len(feature_cols) # number of features

['display_id', 'ad_id', 'clicked', 'uuid', 'events:document_id', 'events:timestamp', 'events:platform', 'events:geo_location', 'ad_prom:document_id', 'ad_prom:campaign_id', 'ad_prom:advertiser_id', 'ad_prom:document_id:source_id', 'ad_prom:document_id:publisher_id', 'ad_prom:document_id:publish_time', 'events:document_id:source_id', 'events:document_id:publisher_id', 'events:document_id:publish_time', 'events:document_id:topic_topI', 'events:document_id:entity_topI', 'events:document_id:categ_topI', 'ad_prom:document_id:topic_topI', 'ad_prom:document_id:entity_topI', 'ad_prom:document_id:categ_topI', 'uuid_hash_code', 'uuid_hash_code_right', 'page_views:uuid:num_visits', 'page_views:uuid:platform-freq', 'page_views:uuid:geo_location-freq', 'page_views:uuid:traffic_source-freq', 'page_views:document_id:source_id-freq', 'page_views:document_id:publisher_id-freq', 'page_views:document_id:topic_topI-freq', 'page_views:document_id:categ_topI-freq', 'page_views:document_id:entity_topI-freq']

['events:document_id',
 'events:platform',
 'events:geo_location',
 'ad_prom:document_id',
 'ad_prom:campaign_id',
 'ad_prom:advertiser_id',
 'ad_prom:document_id:source_id',
 'ad_prom:document_id:publisher_id',
 'events:document_id:source_id',
 'events:document_id:publisher_id',
 'events:document_id:topic_topI',
 'events:document_id:entity_topI',
 'events:document_id:categ_topI',
 'ad_prom:document_id:topic_topI',
 'ad_prom:document_id:entity_topI',
 'ad_prom:document_id:categ_topI',
 'page_views:uuid:platform-freq',
 'page_views:uuid:geo_location-freq',
 'page_views:uuid:traffic_source-freq',
 'page_views:document_id:source_id-freq',
 'page_views:document_id:publisher_id-freq',
 'page_views:document_id:topic_topI-freq',
 'page_views:document_id:categ_topI-freq',
 'page_views:document_id:entity_topI-freq']

In [ ]:
#### Encode f5_all: get cat var keymap #####
label_col = ['clicked'] # label name in f5_all
keymap = OneHotEncoderMapper(f5_all_notdup_uu.loc[:, feature_cols].values) # keymap


In [ ]:
#### Encode f5_all into int values #####
f5_all_notdup_uu.loc[:,feature_cols] = \
    f5_all_notdup_uu.loc[:,feature_cols].apply(lambda x: [keymap[i][x[i]] for i in range(n_feat)], axis = 1) 

In [ ]:
#### check f5_all ####
print f5_all.shape
f5_all.head()

In [ ]:
#### Generate f6_all: convert f5_all into ffm format ####
f6_all = f5_all.copy()
f6_all.loc[:,feature_cols] = f6_all.loc[:,feature_cols].apply(lambda x: \
                                [':'.join(map(str, [i, x[i], 1])) for i in range(n_feat)], axis = 1)

In [ ]:
#### check f6_all ####
print f6_all.shape
f6_all.head()

In [ ]:
#### Save f6_all to file with FFM format ####
output_path = ""
f6_all.loc[:,label_col + feature_cols].to_csv(output_path, sep = ' ', header = False, index = False)

In [ ]:
#### train/val/test split ####
#### set apart label == -1 into test and other into train#### 
#### sample train for a 20 - 80 val-train split #### 


